### Fortran 책 따라하기 

In [74]:
using Dierckx
using Roots

In [75]:
# counter variables
  gamma = 0.5
    beta = 0.99
    alpha = 0.40
    delta = 0.019
    k0 = 10
    # numerical parameters
    k_l = 5
    k_u = 100
    sig = 1e-6
    itermax = 2000
# time path of consumption and capital
TT = 360 # Number of Time paths
k0 = 15

c_t = zeros(TT);
l_t = zeros(TT);
k_t = zeros(TT);
eta_t = zeros(TT);
r_t = zeros(TT);
w_t = zeros(TT);
i_t = zeros(TT);
y_t = zeros(TT);
is_t = zeros(TT);

In [76]:
  # the first order condition
function foc(x_in)
    # x_in 으로 들어오는 것은 현재 period의 소비
    # future capital
    kplus = (1-delta)*k_com + k_com^alpha - x_in

        # calculate future consumption
    cplus = evaluate(spline,kplus)

        # get first order condition
    foc = x_in - (beta*(1+alpha*kplus^(alpha-1)-delta))^(-gamma)*cplus
end 


foc (generic function with 1 method)

In [84]:
NK = 1000
k = range(k_l,k_u, length = NK) ;  
c = k.^alpha - delta.*k ; # Consumption에 대한 초기값
c_new = copy(c);

In [85]:
@time begin
# iterate until policy function converges
for iter in 1:itermax

        # interpolate coefficients
    spline = Dierckx.Spline1D(k,c)

        # calculate decisions for every gridpoint
        for ik = 1:NK

            # set starting value and communicate resource level
            x_in = c[ik]
            k_com = k[ik]

            # find the optimal consumption level
            x_in = try fzero(foc, x_in, order=:secant )
            
            catch x_in
        
        end
            # get optimal consumption and value function
            c_new[ik] = x_in
        end

        # get convergence level
        con_lev = maximum(abs.(c_new - c))
        if iter % 20 == 0
            println("Iteration is $iter"," ", "and error is $con_lev")
        end
        # check for convergence
        if con_lev < sig
            break
        end

        c = copy(c_new)
        if iter == itermax && con_lev > sig
            println("no convergence")
        end
    end
end


Iteration is 20 and error is 0.012747918210263887
Iteration is 40 and error is 0.0043379186884160426
Iteration is 60 and error is 0.002081047028928751
Iteration is 80 and error is 0.0009366118280826896
Iteration is 100 and error is 0.000400089058863351
Iteration is 120 and error is 0.0001672457314265685
Iteration is 140 and error is 6.933683151899572e-5
Iteration is 160 and error is 2.8663712969390076e-5
Iteration is 180 and error is 1.1840168987475863e-5
Iteration is 200 and error is 4.890555779013539e-6
Iteration is 220 and error is 2.02036053575938e-6
  2.764199 seconds (24.35 M allocations: 736.735 MiB, 3.96% gc time)
